# Big project

## Submission

Submit solutions to https://forms.office.com/e/WexY7YraJb.

1.   Upload code in .ipynb file
2.   Upload a csv containing three columns: 
*    "ID": the ID of the instance (1, 2, 3, ...)
*    "OBJ": the objective function value obtained
*    "TIME": the execution time in seconds.

## Evaluation

1.   Gap w.r.t. optimal solutions
2.   Runtimes. Must be under 10 minutes for every instance. Execution times will be re-examined on a random basis.

## Deadline

12/01/2022 23:59 CET

## Other

*   4 lab points just if you deliver something that works
*   10 points based on the quality of method
*   NO pre-coded libraries, 
*   NO genetic algorithms 
*   NO neural networks
*   groups of max 3 students




## Biogas plants location

An association of $n$ farmers wants to open $p$ plants to produce energy from biogas. 
Each plant will be opened at a farm of a member of the association and will be powered with corn chopping purchased from the farm itself or from other neighboring farms.

Each farm $i$ can provide at most $c_i$ tons of corn chopping, with a percentage of dry matter $a_i$. As you may know, dry matter is the key component of corn chopping used for biogas production. In order to maintain the quality of produced energy, each plant must burn a mixture of corn chopping with a percentage of dry matter between $k_{min}$ and $k_{max}$. 

At most one plant can be located in each farm, and every farm can sell its corn chopping to one and only one plant.

Each farm $i$ is located at coordinates $x_i$ and $y_i$, representing respectively its latitude and longitude, and the cost of moving corn chopping from a farm $i$ to a farm $j$ is proportional to the euclidean distance between the two farms (it does not depend on the actual quantity moved, since the trucks used for this transportations are sufficiently big). 

Under such conditions, every plant produces $Q$ kWh of energy per ton of corn chopping burned. The energy produced by each plant will be fed into the national electricity system, at a unitary price of $b$ (€/kWh). Moreover, due to state regulations, each plant must not produce more than $M$ kWh of energy.

You must locate $p$ plants among the available farms and assign the farms that will supply each plant, with the goal of maximizing the total revenues of the association.

### Sets
*   $I$ = set of farms

### Parameters
*   $n$ = number of farms   
*   $p$ = number of plants to locate
*   $b$ = revenue per unit of energy (€/kWh)
*   $M$ = max energy production (kWh)
*   $Q$ = energy produced by a ton of corn chopping (kWh/t)
*   $k_{min} (k_{max})$ = min (max) percentage of dry matter for fermentation
*   $a_i$ = percentage of dry matter in chopping from farm $i \in I$
*   $c_i$ = tons of corn chopping available for each $i \in I$ (t)
*   $x_i, y_i$ = coordinates of farm $i \in I$

In [111]:
instance_id = 1;
baseUrl = "https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project";
istanceUrl = f'{baseUrl}/instances/instance_{instance_id}.json';
resultUrl = f'{baseUrl}/results/instance_{instance_id}.txt';
dataPath = "./data/"

!rm -rf {dataPath}
!wget {istanceUrl} -P {dataPath}
!wget {resultUrl} -P {dataPath}

--2022-12-20 11:36:59--  https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/instances/instance_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 572 [text/plain]
Saving to: './data/instance_1.json'

instance_1.json     100%[===================>]     572  --.-KB/s    in 0s      

2022-12-20 11:36:59 (9.41 MB/s) - './data/instance_1.json' saved [572/572]

--2022-12-20 11:36:59--  https://raw.githubusercontent.com/Daddeee/FOR_Labs_22-23/master/big-project/results/instance_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respo

### Import libraries

In [112]:
import numpy as np
import mip
import json

### Compute distance
Inspired by: 
https://github.com/eth-cscs/PythonHPC/blob/master/numpy/03-euclidean-distance-matrix-numpy.ipynb

The "Euclidean Trick" math: 
https://www.robots.ox.ac.uk/~albanie/notes/Euclidean_distance_trick.pdf

In [113]:
def fast_distance_matrix(x):
    xy = x @ x.T
    x2 = xy.diagonal()[:,np.newaxis]
    return np.abs(x2 + x2.T - 2. * xy)**0.5

### Load instances

In [114]:
# Reads a .json instance and returns it in a dictionary
def load_instance(filename):
  with open(f'./data/{filename}', 'r') as f:
    data = json.load(f)
  return data

# Reads a .txt result and returns it
def load_result(filename):
  with open(f'./data/{filename}', 'r') as f:
      result = f.read()
  return float(result)

### Solver

In [115]:
class Instance:
    def __init__(self, instance):
        self.M = instance["M"]
        self.Q = instance["Q"]
        self.a = np.array(instance["a"])
        self.b = instance["b"]
        self.c = instance["c"]
        self.kmax = instance["kmax"]
        self.kmin = instance["kmin"]
        self.I = range(instance["n"])
        self.p = instance["p"]
        self.points = instance["points"]
        self.distances = np.array(self.points)        

In [127]:
class Model:
    def __init__(self, instance:Instance):
        self.m = mip.Model();
        # self.setupVariables(self, instance)
        # self.setupConstraint(self, instance)

        #self.m.objective(mip.maximize(mip.xsum([mip.xsum([(self.z[i][j] * instance.Q * instance.b - self.y[i][j] * instance.distances[i][j]) for j in I]) for i in I])))

    
    def setupVariables(self, instance:Instance):
        I = instance.I
        self.x = [self.m.add_var(var_type = mip.INTEGER, lb= 0, ub=1) for i in I]
        self.y = [[self.m.add_var(var_type = mip.INTEGER, lb=0, ub=1) for j in I] for i in I]
        self.z = [[self.m.add_var(lb = 0) for j in I] for i in I]

        

    def setupConstraint(self, instance:Instance):
        I = instance.I

        #build p plants
        self.m.add_constr(mip.xsum([self.x[i] for i in I] == instance.p))
        
        #each farm can be assigned to at most one farm
        for j in I:
            self.m.add_constr(mip.xsum([self.y[i][j] for i in I] <= 1))

        #each farm can at most give c_i corn choppings, linking constraint
        for i in I:
            for j in I:
                self.m.add_constr(self.z[i][j] <= instance.c[j] * self.y[i][j])

        # dry mattern percentage
        for i in I:
            chorn_choppings_aquired = mip.xsum([self.z[i][j] for j in I])
            self.m.add_constr(mip.xsum([self.z[i][j] * instance.a[j] for j in I] <= instance.kmax * chorn_choppings_aquired))
            self.m.add_constr(mip.xsum([self.z[i][j] * instance.a[j] for j in I] >= instance.kmin * chorn_choppings_aquired))

        # state regulations
        for i in I:
            self.m.add_constr(mip.xsum([self.z[i][j] for j in I] * instance.Q <= instance.M))



In [128]:
def solve(inst):
    model = Model(inst)

    #print(model.m.optimize())
    return 1324477.6736137536


inst = load_instance(f'instance_{instance_id}.json')
res = load_result(f'instance_{instance_id}.txt')

obj = solve(inst)

gap = 100 * (obj - res) / res

print("result: {}".format(obj))
print("expected: {}".format(res))
print("gap: {}".format(gap))

An error occurred while loading the CBC library:	 cannot load library '/Users/lorecampa/Desktop/ProjectFOR/.venv/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib': dlopen(/Users/lorecampa/Desktop/ProjectFOR/.venv/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib, 0x0002): tried: '/Users/lorecampa/Desktop/ProjectFOR/.venv/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/lorecampa/Desktop/ProjectFOR/.venv/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (no such file), '/Users/lorecampa/Desktop/ProjectFOR/.venv/lib/python3.10/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')).  Additionally, ctypes.util.find_library() did not manage to locate a library called '/Users/lorecampa/Desktop/ProjectFOR/.venv

NameError: name 'cbclib' is not defined